Notebooks to make experiments
-----------------------------

In [38]:
import genotations.ensembl
import polars as pl
from pycomfort.files import *

In [39]:
import sys

base = Path("..")
local = (base / "dashboard").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
else:
    base = Path(".")
%load_ext autoreload
%autoreload 2

[PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', '/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/223.8617.56.plugins/python/helpers-pro/jupyter_debug', '/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/223.8617.56.plugins/python/helpers/pydev', '/home/antonkulaga/sources/expressions-dashboard/notebooks', '/home/antonkulaga/sources/expressions-dashboard', '/home/antonkulaga/micromamba/envs/dashboard/lib/python310.zip', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/s

In [40]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [41]:
from dashboard.cell_models import *
from dashboard.config import *

In [42]:
#expressions = SpeciesExpressions.summaries_from_runs(runs_list)
#expressions

In [43]:
locations = Locations(base)
locations.cell_lines_cache

PosixPath('../data/inputs/cell_lines/CACHE')

In [49]:
cells = Cells(locations.cell_lines)

cell line /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/INS-1 does not seem to exist!


In [50]:
cells.toc_samples

Cell line name,URL,Species,Phenotype,Tissue,Samples,Modified lines,Owners,Sources,Commens
str,str,str,str,str,str,str,str,str,str
"""INS-1""","""https://www.cellosaurus.org/CVCL_0352""","""Rat""","""Celule Beta de insulinom ""","""Insulinom/Pancreas""","""SAMN07421988, SAMN07421989, SAMN07421987, SAMN07421986""",null,"""Molecular Cell Biology Department""",null,null
"""PANC-1""","""https://www.cellosaurus.org/CVCL_0480""","""Rat""","""Beta cells""","""Pancreas""","""SAMN10460093, SAMN06642004""",null,"""Molecular Cell Biology Department""",null,null
"""MelJuSo ""","""https://www.cellosaurus.org/CVCL_1403""","""Human""","""Celule de melanom uman faza VGP""","""Melanoma""","""SAMN07168087, SAMN07168088, SAMN07168089, SAMN07168090, SAMN07168091, SAMN07168092""",null,"""Molecular Cell Biology Department""",null,null
"""HEK293T""","""https://www.cellosaurus.org/CVCL_0063""","""Human""","""Celule umane embrionare de rinichi""","""Human kidney""","""SAMN22420271, SAMN22420269, SAMN22420270""",null,"""Molecular Cell Biology Department, Enzymology Department""",null,null
"""CCD-1070Sk""","""https://www.cellosaurus.org/CVCL_2332""",null,null,null,"""SAMN21374231, SAMN21374232, SAMN21374233, SAMN21374234 """,null,"""Molecular Cell Biology Department""",null,null
"""RAW 264.7""","""https://www.cellosaurus.org/CVCL_0493""","""Mouse""",null,null,"""SAMN32489934, SAMN32489935""",null,"""Molecular Cell Biology Department""",null,"""FAILED, need to redo"""
"""SKOV3""","""https://www.cellosaurus.org/CVCL_0532""","""Human""",null,null,"""SAMN20710030 , SAMN20710031, SAMN20710032 """,null,"""Molecular Cell Biology Department""",null,null
"""B16F10""","""https://www.cellosaurus.org/CVCL_0159""","""Mouse""",null,"""Melanoma""","""SAMN22137224, SAMN22137225, SAMN22137226""",null,"""Ligand-Receptor Interactions Department""",null,null
"""B16F1""","""https://www.cellosaurus.org/CVCL_0158""","""Mouse""",null,null,"""SAMN07340361, SAMN07340362, SAMN07340363""",null,"""Ligand-Receptor Interactions Department""",null,null


In [51]:
summaries = SpeciesExpressions.summaries_from_samples(cells.samples)
summaries

genes summarized for Homo_sapiens: runs 5, merged genes shape (194142, 6)
transcripts summarized for Homo_sapiens: runs 5, merged transcripts shape (194142, 6)
could not find quantification data for /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/B16F10/SAMN22137224/SRR16242848
could not find quantification data for /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/B16F10/SAMN22137224/SRR16242848
could not find quantification data for /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/B16F1/SAMN07340361/SRR5815762
could not find quantification data for /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/B16F1/SAMN07340361/SRR5815762
no quantified transcripts found in SRR16242848;SRR5815762


In [52]:
seq(summaries).map(lambda s: s.species)

['Homo_sapiens', 'Mus_musculus']

In [45]:
SpeciesExpressions.load_summaries_from(locations.cell_lines_cache)

In [ ]:
summaries = SpeciesExpressions.summaries_from_samples(cells.samples)
summaries

## Other stuff

In [46]:
gwas = pl.read_csv("/home/antonkulaga/Downloads/gwas-catalog-v1.0.3-studies-r2022-12-15.tsv", sep="\t")
gwas.head()

DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY,SUMMARY STATS LOCATION,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI
str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""2021-03-25""",32701512,"""Cairns J""","""2020-07-23""","""JCI Insight""","""www.ncbi.nlm.nih.gov/pubmed/32701512""","""Pharmacogenomics of aromatase inhibitors in postmenopausal breast cancer and additional mechanisms of anastrozole action.""","""Estrogen level change in anastrozole-treated breast cancer""","""624 individuals""","""NA""","""Illumina [up to 8000000] (imputed)""",5,"""estrogen measurement, response to aromatase inhibitor""","""http://www.ebi.ac.uk/efo/EFO_0011007, http://purl.obolibrary.org/obo/GO_0061477""","""GCST011407""","""Genome-wide genotyping array""",null,null,null,null,"""estrogen-receptor positive breast cancer""","""http://www.ebi.ac.uk/efo/EFO_1000649"""
"""2020-09-15""",32580138,"""Shi XY""","""2020-06-21""","""EBioMedicine""","""www.ncbi.nlm.nih.gov/pubmed/32580138""","""Identification of susceptibility variants to benign childhood epilepsy with centro-temporal spikes (BECTS) in Chinese Han population.""","""Benign childhood epilepsy with centro-temporal spikes""","""1,738 Han Chinese ancestry cases, 6,592 Han Chinese ancestry controls""","""NA""","""Illumina [5352724] (imputed)""",12,"""Rolandic epilepsy""","""http://www.orpha.net/ORDO/Orphanet_1945""","""GCST010566""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2020-07-09""",32118827,"""Bonde A""","""2020-02-28""","""J Trauma Acute Care Surg""","""www.ncbi.nlm.nih.gov/pubmed/32118827""","""Identification of a new genetic variant associated with cholecystitis: a multicenter genome-wide association study.""","""Acute cholecystitis""","""900 European ancestry cases, 23,727 European ancestry controls""","""NA""","""Illumina [NR] (imputed)""",1,"""Cholecystitis, Acute""","""http://www.ebi.ac.uk/efo/EFO_1001289""","""GCST010176""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2020-07-09""",32118827,"""Bonde A""","""2020-02-28""","""J Trauma Acute Care Surg""","""www.ncbi.nlm.nih.gov/pubmed/32118827""","""Identification of a new genetic variant associated with cholecystitis: a multicenter genome-wide association study.""","""Cholelithiasis""","""up to 2,377 European ancestry cases, 22,203 European ancestry controls""","""NA""","""Illumina [NR] (imputed)""",0,"""cholelithiasis""","""http://www.ebi.ac.uk/efo/EFO_0004799""","""GCST010177""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2021-06-22""",33293549,"""Sargurupremraj M""","""2020-12-08""","""Nat Commun""","""www.ncbi.nlm.nih.gov/pubmed/33293549""","""Cerebral small vessel disease genomics and its implications across the lifespan.""","""White matter hyperintensity volume x hypertension interaction (2df)""","""46,055 European ancestry individuals""","""NA""","""Affymetrix, Illumina [9900000] (imputed)""",26,"""hypertension, white matter hyperintensity measurement""","""http://www.ebi.ac.uk/efo/EFO_0000537, http://www.ebi.ac.uk/efo/EFO_0005665""","""GCST011953""","""Genome-wide genotyping array""",null,null,null,null,null,null


In [47]:
traits = gwas\
    .groupby([pl.col("MAPPED_TRAIT").alias("trait"), pl.col("MAPPED_TRAIT_URI").alias("trait_uri")])\
    .agg([pl.count("STUDY ACCESSION").alias("studies"), pl.sum("ASSOCIATION COUNT").alias("association_counts"), pl.count("PUBMED ID").alias("papers")])\
    .sort(["studies","association_counts", "papers"], True)
traits

trait,trait_uri,studies,association_counts,papers
str,str,u32,i64,u32
"""blood protein measurement""","""http://www.ebi.ac.uk/efo/EFO_0007937""",5354,8587,5354
"""protein measurement""","""http://www.ebi.ac.uk/efo/EFO_0004747""",5242,3814,5242
"""metabolite measurement""","""http://www.ebi.ac.uk/efo/EFO_0004725""",3040,2776,3040
"""gut microbiome measurement""","""http://www.ebi.ac.uk/efo/EFO_0007874""",2300,6547,2300
"""functional brain measurement""","""http://www.ebi.ac.uk/efo/EFO_0007849""",1811,147,1811
"""brain connectivity measurement""","""http://www.ebi.ac.uk/efo/EFO_0005210""",1706,100,1706
"""phosphatidylcholine measurement""","""http://www.ebi.ac.uk/efo/EFO_0010226""",733,4805,733
"""clinical treatment""","""http://www.ebi.ac.uk/efo/EFO_0007056""",687,31,687
"""employment status""","""http://www.ebi.ac.uk/efo/EFO_0005241""",687,10,687


In [ ]:
traits.write_csv("/home/antonkulaga/Downloads/TRAITS.tsv", sep="\t")

In [ ]:
no_measure = traits.filter(~pl.col("trait").str.contains("measurement") & ~pl.col("trait").str.contains("count"))
    no_measure .write_csv("/home/antonkulaga/Downloads/traits_no_measurement.tsv",  sep="\t")
no_measure#%%
import polars as pl
from pathlib import Path
from pycomfort.files import *
from pycomfort import files
import pyarrow
import pandas as pd
from functional import seq
from typing import *
import functools

In [ ]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

In [ ]:
gwas = pl.read_csv("/home/antonkulaga/Downloads/gwas-catalog-v1.0.3-studies-r2022-12-15.tsv", sep="\t")
gwas.head()

In [ ]:
traits = gwas\
    .groupby([pl.col("MAPPED_TRAIT").alias("trait"), pl.col("MAPPED_TRAIT_URI").alias("trait_uri")])\
    .agg([pl.count("STUDY ACCESSION").alias("studies"), pl.sum("ASSOCIATION COUNT").alias("association_counts"), pl.count("PUBMED ID").alias("papers")])\
    .sort(["studies","association_counts", "papers"], True)
traits

In [ ]:
traits.write_csv("/home/antonkulaga/Downloads/TRAITS.tsv", sep="\t")

In [20]:
no_measure = traits.filter(~pl.col("trait").str.contains("measurement") & ~pl.col("trait").str.contains("count"))
no_measure .write_csv("/home/antonkulaga/Downloads/traits_no_measurement.tsv",  sep="\t")
no_measure.shape

(3325, 5)